In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import logging

In [3]:
# Configuring logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Image Processing Functions

In [4]:
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def remove_noise(image, kernel_size=(7, 7)):
    return cv2.GaussianBlur(image, kernel_size, 0)


def increase_contrast(image):
    min_val, max_val = np.min(image), np.max(image)
    stretched = ((image - min_val) / (max_val - min_val) * 300).clip(0, 255).astype(np.uint8)
    return stretched


def apply_hpf(image):
    kernel = np.array([[-2, -2, -2],
                       [-2,  16, -2],
                       [-2, -2, -2]])  
    hpf_image = cv2.filter2D(image, -1, kernel)
    
    # Normalizing
    hpf_image = cv2.normalize(hpf_image, None, 0, 255, cv2.NORM_MINMAX)
    
    return hpf_image.astype(np.uint8)


def marr_hildreth_edge_detection(image, sigma=2.0):
    
    # Gaussian Blur
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    
    # Laplacian
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    
    # Finding zero-crossings
    zero_crossings = np.zeros_like(laplacian, dtype=np.uint8)
    laplacian_padded = np.pad(laplacian, ((1, 1), (1, 1)), mode='constant')
    
    for i in range(1, laplacian.shape[0] + 1):
        for j in range(1, laplacian.shape[1] + 1):
            neighbors = laplacian_padded[i-1:i+2, j-1:j+2]
            if (laplacian_padded[i, j] > 0 and np.min(neighbors) < 0) or (laplacian_padded[i, j] < 0 and np.max(neighbors) > 0):
                zero_crossings[i-1, j-1] = 255
    return zero_crossings


def canny_edge_detection(image, low_threshold, high_threshold):
    return cv2.Canny(image, low_threshold, high_threshold)


def segment_coins(image, edges, min_radius_ratio=0.23, max_radius_ratio=0.7):
    
    print(f"Image shape: {image.shape}, Edges shape: {edges.shape}")
  
    height, width = image.shape[:2]
    min_radius = int(min_radius_ratio * min(height, width))
    max_radius = int(max_radius_ratio * min(height, width))
    
    if edges.shape[:2] != (height, width):
        print("Resizing edges to match image dimensions")
        edges = cv2.resize(edges, (width, height))
    
    # Finding contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Creating mask
    segmentation_mask = np.zeros((height, width), dtype=np.uint8)
    detection_image = image.copy()
    
    segmented_coins = []
    coin_masks = []
    valid_contours = []
    
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > 0:
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            
            if min_radius <= radius <= max_radius:
                valid_contours.append(cnt)
                
                center = (int(x), int(y))
                radius_int = int(radius)
                cv2.circle(detection_image, center, radius_int, (0, 255, 0), 2)
                
                coin_mask = np.zeros((height, width), dtype=np.uint8)
                cv2.circle(coin_mask, center, radius_int, 255, -1)
                
                segmentation_mask[coin_mask > 0] = i+1
                
                individual_coin = np.zeros_like(image)
                individual_coin = image.copy()
                individual_coin[coin_mask == 0] = 0
                
                x_min = max(0, int(x - radius) - 10)
                y_min = max(0, int(y - radius) - 10)
                x_max = min(width, int(x + radius) + 10)
                y_max = min(height, int(y + radius) + 10)
                
                if x_min < x_max and y_min < y_max:
                    try:
                        cropped_coin = individual_coin[y_min:y_max, x_min:x_max]
                        cropped_mask = coin_mask[y_min:y_max, x_min:x_max]
                        
                        segmented_coins.append(cropped_coin)
                        coin_masks.append(cropped_mask)
                    except Exception as e:
                        print(f"Error cropping coin: {e}")
    
    colored_segmentation = np.zeros((height, width, 3), dtype=np.uint8)
    
    for i in range(1, len(valid_contours) + 1):
        color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))
        mask = (segmentation_mask == i)
        if np.any(mask):  
            colored_segmentation[mask] = color
    
    return detection_image, segmentation_mask, colored_segmentation, segmented_coins, coin_masks, len(valid_contours)


Image Processing Pipeline

In [5]:
def process_image(image_path, output_folder, params):
    
    try:
        
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image at {image_path} could not be read.")
        
        results = {
            'original': image.copy(),
            'original_rgb': cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        }
        
        # Converting to grayscale
        results['gray'] = convert_to_grayscale(image)

        # Removing noise
        results['denoised'] = remove_noise(results['gray'], params['blur_kernel'])

        # Contrast stretching
        results['contrast'] = increase_contrast(results['denoised'])

        # Applying High-Pass Filter (HPF)
        results['hpf'] = apply_hpf(results['contrast'])


        # Applying Marr-Hildreth edge detection
        results['edges_marr_hildreth'] = marr_hildreth_edge_detection(
            results['hpf'], params['marr_hildreth_sigma']
        )
        
        (results['detection_marr_hildreth'], 
         results['mask_marr_hildreth'],
         results['colored_mask_marr_hildreth'],
         results['segmented_coins_marr_hildreth'],
         results['coin_masks_marr_hildreth'],
         results['count_marr_hildreth']) = segment_coins(
            image.copy(), results['edges_marr_hildreth'], 
            params['min_radius_ratio'], params['max_radius_ratio']
        )


        # Applying Canny edge detection
        results['edges_canny'] = canny_edge_detection(
            results['hpf'], params['canny_low'], params['canny_high']
        )
        
        (results['detection_canny'], 
         results['mask_canny'],
         results['colored_mask_canny'],
         results['segmented_coins_canny'],
         results['coin_masks_canny'],
         results['count_canny']) = segment_coins(
            image.copy(), results['edges_canny'],
            params['min_radius_ratio'], params['max_radius_ratio']
        )

        # Saving detection and segmentation results
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        
        segmentation_dir = os.path.join(output_folder, base_name + "_segmented")
        os.makedirs(segmentation_dir, exist_ok=True)
        
        # Saving Marr-Hildreth results
        cv2.imwrite(os.path.join(output_folder, f"{base_name}_marr_hildreth_detection.jpg"), 
                    results['detection_marr_hildreth'])
        cv2.imwrite(os.path.join(output_folder, f"{base_name}_marr_hildreth_segmentation.jpg"), 
                    results['colored_mask_marr_hildreth'])
        
        for i, coin in enumerate(results['segmented_coins_marr_hildreth']):
            cv2.imwrite(os.path.join(segmentation_dir, f"coin_marr_hildreth_{i+1}.jpg"), coin)
        
        
        # Saving Canny results
        cv2.imwrite(os.path.join(output_folder, f"{base_name}_canny_detection.jpg"), 
                    results['detection_canny'])
        cv2.imwrite(os.path.join(output_folder, f"{base_name}_canny_segmentation.jpg"), 
                    results['colored_mask_canny'])
        
        for i, coin in enumerate(results['segmented_coins_canny']):
            cv2.imwrite(os.path.join(segmentation_dir, f"coin_canny_{i+1}.jpg"), coin)
        
        
        logging.info(f"{base_name}: Marr-Hildreth detected and segmented {results['count_marr_hildreth']} coins.")
        logging.info(f"{base_name}: Canny detected and segmented {results['count_canny']} coins.")
        
        return results, base_name
        
    except Exception as e:
        logging.error(f"Error processing {os.path.basename(image_path)}: {e}")
        return None, None


Visualization

In [6]:
def create_visualization(results, base_name, vis_folder):
    """Create and save visualization of all processing steps including segmentation."""
    try:
        if results is None:
            return None
            
        plt.figure(figsize=(20, 15))
        
        # Original Image
        plt.subplot(3, 4, 1)
        plt.imshow(results['original_rgb'])
        plt.title('1. Original Image')
        plt.axis('off')
        
        # Grayscale
        plt.subplot(3, 4, 2)
        plt.imshow(results['gray'], cmap='gray')
        plt.title('2. Grayscale')
        plt.axis('off')
        
        # Denoised
        plt.subplot(3, 4, 3)
        plt.imshow(results['denoised'], cmap='gray')
        plt.title('3. Denoised (Gaussian Blur)')
        plt.axis('off')
        
        # Contrast Enhanced
        plt.subplot(3, 4, 4)
        plt.imshow(results['contrast'], cmap='gray')
        plt.title('4. Contrast Enhanced')
        plt.axis('off')
        
        # High-Pass Filter
        plt.subplot(3, 4, 5)
        plt.imshow(results['hpf'], cmap='gray')
        plt.title('5. High-Pass Filter')
        plt.axis('off')
        
        # Marr-Hildreth Edges
        plt.subplot(3, 4, 6)
        plt.imshow(results['edges_marr_hildreth'], cmap='gray')
        plt.title('6. Marr-Hildreth Edges')
        plt.axis('off')
        
        # Marr-Hildreth Detection Results
        plt.subplot(3, 4, 7)
        plt.imshow(cv2.cvtColor(results['detection_marr_hildreth'], cv2.COLOR_BGR2RGB))
        plt.title(f'8. Marr-Hildreth Detection\n({results["count_marr_hildreth"]} coins)')
        plt.axis('off')
        
        # Marr-Hildreth Segmentation Mask
        plt.subplot(3, 4, 8)
        plt.imshow(results['mask_marr_hildreth'], cmap='nipy_spectral')
        plt.title('9. Marr-Hildreth Segmentation Mask')
        plt.axis('off')
        
        # Marr-Hildreth Colored Segmentation
        plt.subplot(3, 4, 9)
        plt.imshow(cv2.cvtColor(results['colored_mask_marr_hildreth'], cv2.COLOR_BGR2RGB))
        plt.title('10. Marr-Hildreth Colored Segmentation')
        plt.axis('off')
        
        # Canny Edges
        plt.subplot(3, 4, 10)
        plt.imshow(results['edges_canny'], cmap='gray')
        plt.title('7. Canny Edges')
        plt.axis('off')
        
        # Canny Detection Results
        plt.subplot(3, 4, 11)
        plt.imshow(cv2.cvtColor(results['detection_canny'], cv2.COLOR_BGR2RGB))
        plt.title(f'11. Canny Detection\n({results["count_canny"]} coins)')
        plt.axis('off')
        
        # Canny Colored Segmentation
        plt.subplot(3, 4, 12)
        plt.imshow(cv2.cvtColor(results['colored_mask_canny'], cv2.COLOR_BGR2RGB))
        plt.title('12. Canny Colored Segmentation')
        plt.axis('off')
        
     
        plt.tight_layout()
        
        # Saving the visualization
        visualization_path = os.path.join(vis_folder, f"{base_name}_visualization.png")
        plt.savefig(visualization_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        
        # Visualization for showing individual segmented coins
        
        # Marr-Hildreth 
        if len(results['segmented_coins_marr_hildreth']) > 0:
            n_coins = len(results['segmented_coins_marr_hildreth'])
            rows = min(4, n_coins)
            cols = int(np.ceil(n_coins / rows))
            
            plt.figure(figsize=(15, 10))
            for i, coin in enumerate(results['segmented_coins_marr_hildreth']):
                if i < rows * cols:
                    plt.subplot(rows, cols, i + 1)
                    plt.imshow(cv2.cvtColor(coin, cv2.COLOR_BGR2RGB))
                    plt.title(f'Coin {i+1} (Marr-Hildreth)')
                    plt.axis('off')
            
            plt.tight_layout()
            mh_coins_path = os.path.join(vis_folder, f"{base_name}_marr_hildreth_coins.png")
            plt.savefig(mh_coins_path, dpi=300, bbox_inches='tight')
            plt.close()
            logging.info(f"Marr-Hildreth visualization created for {base_name}")

        # Canny 
        if len(results['segmented_coins_canny']) > 0:
            n_coins = len(results['segmented_coins_canny'])
            rows = min(4, n_coins)
            cols = int(np.ceil(n_coins / rows))
            
            plt.figure(figsize=(15, 10))
            for i, coin in enumerate(results['segmented_coins_canny']):
                if i < rows * cols:
                    plt.subplot(rows, cols, i + 1)
                    plt.imshow(cv2.cvtColor(coin, cv2.COLOR_BGR2RGB))
                    plt.title(f'Coin {i+1} (Canny)')
                    plt.axis('off')
            
            plt.tight_layout()
            canny_coins_path = os.path.join(vis_folder, f"{base_name}_canny_coins.png")
            plt.savefig(canny_coins_path, dpi=300, bbox_inches='tight')
            plt.close()
            logging.info(f"Canny visualization created for {base_name}")


        if len(results['segmented_coins_marr_hildreth']) == 0 and len(results['segmented_coins_canny']) == 0:
            logging.warning(f"No coins detected in {base_name} by either method")

        logging.info(f"Visualization process completed for {base_name}")
        return visualization_path
    
    except Exception as e:
        logging.error(f"Error creating visualization for {base_name}: {e}")
        return None
    


In [7]:
def main(input_folder, output_folder, vis_folder, params=None):
    
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(vis_folder, exist_ok=True)
 
    if params is None:
        params = {
            'blur_kernel': (7, 7),
            'marr_hildreth_sigma': 2.0,
            'canny_low': 50,
            'canny_high': 150,
            'min_radius_ratio': 0.1,
            'max_radius_ratio': 0.7
        }
    
    for image_name in os.listdir(input_folder):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, image_name)
            
            results, base_name = process_image(image_path, output_folder, params)
            
            if results:
                create_visualization(results, base_name, vis_folder)
    
    logging.info(f"Processing complete. Results saved in {output_folder}")
    logging.info(f"Visualizations saved in {vis_folder}")


In [17]:
if __name__ == "__main__":
    input_folder = "input_images"
    output_folder = "detection_results"
    vis_folder = "visualization_results"
    
    params = {
        'blur_kernel': (7, 7),          # Gaussian blur kernel size
        'marr_hildreth_sigma': 2.0,     # Sigma for Marr-Hildreth edge detection
        'canny_low': 100,               # Lower threshold for Canny edge detection
        'canny_high': 300,              # Upper threshold for Canny edge detection
        'min_radius_ratio': 0.24,       # Minimum coin radius as ratio of image dimension
        'max_radius_ratio': 0.7         # Maximum coin radius as ratio of image dimension
    }
    
    main(input_folder, output_folder, vis_folder, params)

C:\Users\rishi\AppData\Local\Temp\ipykernel_20780\4160640620.py:89: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 370 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  segmentation_mask[coin_mask > 0] = i+1
C:\Users\rishi\AppData\Local\Temp\ipykernel_20780\4160640620.py:89: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 884 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  segmentation_mask[coin_mask > 0] = i+1
2025-03-02 22:59:46,105 - INFO - 1: Marr-Hildreth detected and segmented 0 coins.
2025-03-02 22:59:46,106 - INFO - 1: Canny detected and segmented 2 coins.


Image shape: (481, 496, 3), Edges shape: (481, 496)
Image shape: (481, 496, 3), Edges shape: (481, 496)


2025-03-02 22:59:51,307 - INFO - Canny visualization created for 1
2025-03-02 22:59:51,308 - INFO - Visualization process completed for 1
2025-03-02 22:59:51,422 - INFO - 10: Marr-Hildreth detected and segmented 1 coins.
2025-03-02 22:59:51,422 - INFO - 10: Canny detected and segmented 2 coins.


Image shape: (163, 308, 3), Edges shape: (163, 308)
Image shape: (163, 308, 3), Edges shape: (163, 308)


2025-03-02 22:59:53,307 - INFO - Marr-Hildreth visualization created for 10
2025-03-02 22:59:53,549 - INFO - Canny visualization created for 10
2025-03-02 22:59:53,549 - INFO - Visualization process completed for 10
2025-03-02 22:59:53,655 - INFO - 2: Marr-Hildreth detected and segmented 1 coins.
2025-03-02 22:59:53,655 - INFO - 2: Canny detected and segmented 3 coins.


...

2025-03-02 23:00:29,669 - INFO - Marr-Hildreth visualization created for 7
2025-03-02 23:00:30,140 - INFO - Canny visualization created for 7
2025-03-02 23:00:30,141 - INFO - Visualization process completed for 7
2025-03-02 23:00:30,345 - INFO - 8: Marr-Hildreth detected and segmented 1 coins.
2025-03-02 23:00:30,346 - INFO - 8: Canny detected and segmented 1 coins.


Image shape: (408, 612, 3), Edges shape: (408, 612)
Image shape: (408, 612, 3), Edges shape: (408, 612)


2025-03-02 23:00:33,138 - INFO - Marr-Hildreth visualization created for 8
2025-03-02 23:00:33,470 - INFO - Canny visualization created for 8
2025-03-02 23:00:33,471 - INFO - Visualization process completed for 8
C:\Users\rishi\AppData\Local\Temp\ipykernel_20780\4160640620.py:89: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 657 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  segmentation_mask[coin_mask > 0] = i+1
C:\Users\rishi\AppData\Local\Temp\ipykernel_20780\4160640620.py:89: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 881 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  segmentation_mask[coin_mask > 0] = i+1
C:\Users\

Image shape: (307, 612, 3), Edges shape: (307, 612)
Image shape: (307, 612, 3), Edges shape: (307, 612)


2025-03-02 23:00:36,856 - INFO - Marr-Hildreth visualization created for 9
2025-03-02 23:00:37,192 - INFO - Canny visualization created for 9
2025-03-02 23:00:37,192 - INFO - Visualization process completed for 9
2025-03-02 23:00:37,194 - INFO - Processing complete. Results saved in detection_results
2025-03-02 23:00:37,194 - INFO - Visualizations saved in visualization_results
